In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt  
import datetime
from datetime import datetime,timedelta
import time
from pandas.tseries.offsets import MonthEnd
from os import listdir
from os.path import isfile, join
import math

In [2]:
from pandas.plotting import scatter_matrix
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
import seaborn as sns

In [3]:
def read_csv(name):
    data=pd.read_csv("path/"+name+".csv")
    return data

def read_excel(name,sheet_name):
    data=pd.read_excel("path/"+name+".xlsx",sheet_name=sheet_name)
    return data

def save(data,name):
    data.to_csv("path/data/"+name+".csv")
    
def getlist(df,col):
    acc_list=list(df[col].drop_duplicates())
    return acc_list

def loc_one(df,col,item):
    loc_df=df.loc[df[col]==item]
    return loc_df

def loc_list(df,col,acc_list):
    loc_df=df.loc[df[col].isin(acc_list)]
    return loc_df

In [57]:
# filter data by segment
data=read_csv('filtered_data6')
segment_info=read_csv('teamid with names')
smb_list=getlist(loc_one(segment_info,'segment','SMB'),'teamid')
mm_list=getlist(loc_one(segment_info,'segment','MM'),'teamid')
ent_list=getlist(loc_one(segment_info,'segment','ENT'),'teamid')

seg_list=smb_list
fil_data=loc_list(data,'teamid

In [58]:
### ONE find the bottom 5% view level for all users
fil_data=read_csv('filtered_data6')


data=fil_data
name_dict=dict(zip(data.teamid,data.company
acc_list=getlist(data,'team_id')

#1 combine all files into 1 table & filter with current acc list
mypath='path...'
file_names = [f for f in listdir(mypath) if isfile(join(mypath, f))]
user_data=pd.DataFrame()
for file in file_names:
    df=pd.read_csv(mypath+'/'+file)
    user_data=user_data.append(df)
#save(user_data.'combined_df')
#user_data=read_csv('combined_df')
all_data=loc_list(user_data,'team_id',acc_list)
all_data=loc_one(all_data,'team_report_type','function2')

#2 get monthly data for each user
all_data['date']=pd.to_datetime(all_data['date'],format='%Y-%m-%d %H:%M:%S')
month_view=all_data.groupby(['userid',pd.Grouper(key='date',freq='M')])['value'].sum().reset_index()
user_avg_view=month_view.groupby(['userid'])['value'].mean().reset_index()
usage=list(user_avg_view['value'])

#3 get 5% quantile
bottom_quantiles=user_avg_view.quantile([0.05,0.1,0.25,0.3])
bottom_25=user_avg_view.quantile([0.25]).reset_index()
low_view=bottom_25['value'][0]
print(low_view)

### TW0 filter those with accounts with consistent low use
#1 count num of user in each account
num_of_users=all_data.groupby('teamid')['userid'].nunique()
user_num_dict=num_of_users.to_dict()

#2 filter the accounts using benchmark set by number of users and 25%quantile 
problems=[]
consistent_low_list=[]
avg_low_list=[]

for acc in acc_list:
    acc_df=loc_one(data,'teamid',acc)
    view=list(acc_df['function2'])
    view.remove(max(view))
    try:
        low_use=user_num_dict[acc]*low_view
        avg_view=sum(view)/len(view)
        high_view = [i for i in view if i > low_use]
        if high_view == []:
            consistent_low_list.append(acc)
            #consistent_low_df=consistent_low_df.append(com_df)
        if avg_view <= low_use:
            avg_low_list.append(acc)
            #avg_low_df=low_use_df.append(com_df)
    except:
        problems.append(acc)
problem_df=loc_list(data,'teamid',problems)
consis_low_df=loc_list(data,'teamid',consistent_low_list)
avg_low_df=loc_list(data,'teamid',avg_low_list)
#save(problem_df,'new_customers')
#save(low_use_df,'low_use')

19.555555555555557


In [59]:
### filter sudden drop - initial

drop_pct=0.9

#fil_data=read_csv('filtered_data')
data=fil_data
name_dict=dict(zip(data.teamid,data.company))
acc_list=getlist(data,'teamid')

#1 for those accs with >= 4 months, identify drop based on first 3 months of data, put them into dicts
drop_mon_dict=dict()
drop_amount_dict=dict()
rise_amount_dict=dict()
total_month_dict=dict()
first_drop_month=dict()
first_drop_date=dict()
zero_use=[]

for acc in acc_list:
    acc_df=loc_one(data,'teamid',acc)
    view_list=list(acc_df['function2'])
    date_list=list(acc_df['date'])
    initial_view= sum(view_list[0:3])/3
    #initial_view_list= view_list[0:3]
    #low_use=user_dict[com]*low_view
    #low_initial_view=[i for i in initial_view_list if i <= low_use]  
    #if low_initial_view == []:
    if initial_view != 0:
        num_of_drop=0
        total_months=len(view_list)
        month_of_drop=[]
        date_of_drop=[]
        for i in range(len(view_list)-3):
            month_view=view_list[i+3]
            date=date_list[i+3]
            change = (initial_view-month_view)/initial_view
            if change >= drop_pct:
                num_of_drop+=1
                month_of_drop.append(i+4)
                date_of_drop.append(date)
        num_of_rise=total_months-num_of_drop-3
        total_month_dict[acc]=total_months
        drop_amount_dict[acc]=num_of_drop
        rise_amount_dict[acc]=num_of_rise
        try:
            first_drop_date[acc]=date_of_drop[0]
        except:
            first_drop_date[acc]=np.nan
        try:
            first_drop_month[acc]=month_of_drop[0]
        except:
            first_drop_month[acc]=np.nan
        
    else:
        zero_use.append(acc)

  
 #2 covert dicts to dataframe
drop_df = pd.DataFrame(list(total_month_dict.items()),columns = ['team_id','total_months'])
drop_mon=pd.DataFrame(list(drop_amount_dict.items()),columns = ['team_id','drop_mon'])
rise_mon=pd.DataFrame(list(rise_amount_dict.items()),columns = ['team_id','rise_mon'])
first_drop=pd.DataFrame(list(first_drop_month.items()),columns = ['team_id','first_drop_mon'])
first_date=pd.DataFrame(list(first_drop_date.items()),columns = ['team_id','first_drop_date'])
drop_df=pd.merge(drop_df,drop_mon,on='team_id')
drop_df=pd.merge(drop_df,rise_mon,on='team_id')
drop_df=pd.merge(drop_df,first_drop,on='team_id')
drop_df=pd.merge(drop_df,first_date,on='team_id')

name_df=pd.DataFrame(list(name_dict.items()),columns = ['team_id','team_name'])
drop_df=pd.merge(drop_df,name_df,on='team_id')
drop_df=drop_df.replace('', 0) 


#3 filter those with more drop than rise, rising months of 1 or less
drop_df['first_drop_mon'] = drop_df['first_drop_mon'].replace({0:np.nan})
drop_df['after_rise_mon']=drop_df['rise_mon']-(drop_df['first_drop_mon']-4)

sudden_drop_info=drop_df.loc[drop_df['drop_mon'] > drop_df['after_rise_mon']]
sudden_drop_info=sudden_drop_info.loc[sudden_drop_info['after_rise_mon']<=1]

drop_com_list=getlist(sudden_drop_info,'team_id')
sudden_drop_df=loc_list(data,'teamid',drop_com_list)
#save(sudden_drop_df,'sudden_drop_90_overlapped')
#save(drop_df,'drop_info_90')


#4 get common accounts from drop_df and low_use_df
common_acc=list(set(drop_com_list)&set(consistent_low_list))

#5 for the sudden drop accounts,filter out those with sudden drop in first 3 months as they are fluctuating accounts
data=sudden_drop_df
drop_three_list=list()
for com in drop_com_list:
#for a in range(1):
    #com='team5da5fcf33fa784.09589832'
    com_df=loc_one(data,'teamid',com)
    view_list=list(com_df['function2'])
    initial_view=view_list[0]
    if initial_view != 0:
        for i in range(2):
            month_view=view_list[i+1]
            change=(initial_view-month_view)/initial_view
            if change >= drop_pct:
                drop_three_list.append(com)
                break
            else:
                continue
    else:
        initial_view=view_list[1]
        if initial_view != 0:
            third_view=view_list[2]
            change=(initial_view-third_view)/initial_view
            if change >= drop_pct:
                drop_three_list.append(com)
            
drop_three_df=loc_list(data,'teamid',drop_three_list)

#6 remove the accs with sudden drop in first 3 months as they are fluctuating accounts from the sudden drop 
non_three_list=list(set(drop_com_list)-set(drop_three_list))
sudden_drop_df=loc_list(sudden_drop_df,'teamid',non_three_list)
sudden_drop_list=getlist(sudden_drop_df,'teamid')

In [127]:
### filter sudden drop - max

drop_pct=0.9
bounce=1

#data=read_csv('filtered_data')
data=fil_data
name_dict=dict(zip(data.teamid,data.company))
acc_list=getlist(data,'teamid')

#1 for those accs with >= 4 months, identify drop based on first 3 months of data, put them into dicts
drop_mon_dict=dict()
drop_amount_dict=dict()
rise_amount_dict=dict()
total_month_dict=dict()
first_drop_month=dict()
max_month_dict=dict()
zero_use=[]

for acc in acc_list:
    acc_df=loc_one(data,'teamid',acc)
    view_list=list(acc_df['function2'])
    date_list=list(acc_df['date'])
    max_view= max(view_list)
    max_month=view_list.index(max_view)+1
    if max_view != 0:
        num_of_drop=0
        total_months=len(view_list)
        month_of_drop=[]
        date_of_drop=[]
        for i in range(len(view_list)-max_month):
            month_view=view_list[i+max_month]
            date=date_list[i+max_month]
            change = (max_view-month_view)/max_view
            if change >= drop_pct:
                num_of_drop+=1
                month_of_drop.append(i+max_month+1)
                date_of_drop.append(date)
        num_of_rise=total_months-num_of_drop-max_month
        total_month_dict[acc]=total_months
        drop_amount_dict[acc]=num_of_drop
        rise_amount_dict[acc]=num_of_rise
        max_month_dict[acc]=max_month
        try:
            first_drop_date[acc]=date_of_drop[0]
        except:
            first_drop_date[acc]=np.nan
        try:
            first_drop_month[acc]=month_of_drop[0]
        except:
            first_drop_month[acc]=np.nan        
    else:
        zero_use.append(acc)
        
 #2 covert dicts to dataframe
drop_df = pd.DataFrame(list(total_month_dict.items()),columns = ['team_id','total_months'])
drop_mon=pd.DataFrame(list(drop_amount_dict.items()),columns = ['team_id','drop_mon'])
rise_mon=pd.DataFrame(list(rise_amount_dict.items()),columns = ['team_id','rise_mon'])
first_drop=pd.DataFrame(list(first_drop_month.items()),columns = ['team_id','first_drop_mon'])
first_date=pd.DataFrame(list(first_drop_date.items()),columns = ['team_id','first_drop_date'])
max_month=pd.DataFrame(list(max_month_dict.items()),columns = ['team_id','max_month'])
drop_df=pd.merge(drop_df,drop_mon,on='team_id')
drop_df=pd.merge(drop_df,rise_mon,on='team_id')
drop_df=pd.merge(drop_df,first_drop,on='team_id')
drop_df=pd.merge(drop_df,first_date,on='team_id')
drop_df=pd.merge(drop_df,max_month,on='team_id')

name_df=pd.DataFrame(list(name_dict.items()),columns = ['team_id','team_name'])
drop_df=pd.merge(drop_df,name_df,on='team_id')
drop_df=drop_df.replace('', 0) 


#3 filter those with more drop than rise, rising months of 1 or less
drop_df['first_drop_mon'] = drop_df['first_drop_mon'].replace({0:np.nan})
drop_df['after_rise_mon']=drop_df['rise_mon']-(drop_df['first_drop_mon']-drop_df['max_month']-1)

sudden_drop_info=drop_df.loc[drop_df['drop_mon'] > drop_df['after_rise_mon']]
sudden_drop_info=sudden_drop_info.loc[sudden_drop_info['after_rise_mon']<=bounce]

sudden_drop_list=getlist(sudden_drop_info,'team_id')
sudden_drop_df=loc_list(data,'teamid',drop_com_list)
#save(sudden_drop_df,'sudden_drop_90_overlapped')
#save(drop_df,'drop_info_90')


#4 get common accounts from drop_df and low_use_df
common_acc=list(set(drop_com_list)&set(consistent_low_list))


In [130]:
### filter sudden drop - with higher after_rise benchmark

drop_pct=0.9
bounce=1

#data=read_csv('filtered_data')
data=fil_data
name_dict=dict(zip(data.teamid,data.company))
acc_list=getlist(data,'teamid')
#1 for those accs with >= 4 months, identify drop based on first 3 months of data, put them into dicts
drop_mon_dict=dict()
drop_amount_dict=dict()
rise_amount_dict=dict()
total_month_dict=dict()
rebounce_dict=dict()
first_drop_month=dict()
first_drop_date=dict()
max_month_dict=dict()
zero_use=[]

for acc in acc_list:
    acc_df=loc_one(data,'teamid',acc)
    view_list=list(acc_df['function2'])
    date_list=list(acc_df['date'])
    max_view= max(view_list)
    max_month=view_list.index(max_view)+1
    drop_month=0
    if max_view != 0:
        num_of_drop=0
        rebounce=0
        total_months=len(view_list)
        month_of_drop=[]
        date_of_drop=[]
        for i in range(len(view_list)-max_month):
            month_view=view_list[i+max_month]
            date=date_list[i+max_month]
            change = (max_view-month_view)/max_view
            if change >= drop_pct:
                num_of_drop+=1
                drop_month=i+max_month+1
                month_of_drop.append(drop_month)
                break
        drop_num=drop_month
        total_month_dict[acc]=total_months
        drop_amount_dict[acc]=num_of_drop
        for i in range(len(view_list)-drop_num):
            month_view=view_list[i+drop_num]
            if month_view > 0.5*max_view:
                rebounce+=1
        rebounce_dict[acc]=rebounce
        max_month_dict[acc]=max_month
        try:
            first_drop_date[acc]=date_of_drop[0]
        except:
            first_drop_date[acc]=np.nan
        try:
            first_drop_month[acc]=month_of_drop[0]
        except:
            first_drop_month[acc]=np.nan        
    else:
        zero_use.append(acc)
        
 #2 covert dicts to dataframe
drop_df = pd.DataFrame(list(total_month_dict.items()),columns = ['team_id','total_months'])
drop_mon=pd.DataFrame(list(drop_amount_dict.items()),columns = ['team_id','drop_mon'])
rebounce=pd.DataFrame(list(rebounce_dict.items()),columns = ['team_id','rebounce'])
first_drop=pd.DataFrame(list(first_drop_month.items()),columns = ['team_id','first_drop_mon'])
first_date=pd.DataFrame(list(first_drop_date.items()),columns = ['team_id','first_drop_date'])
max_month=pd.DataFrame(list(max_month_dict.items()),columns = ['team_id','max_month'])
drop_df=pd.merge(drop_df,drop_mon,on='team_id')
drop_df=pd.merge(drop_df,rebounce,on='team_id')
drop_df=pd.merge(drop_df,first_drop,on='team_id')
drop_df=pd.merge(drop_df,first_date,on='team_id')
drop_df=pd.merge(drop_df,max_month,on='team_id')

name_df=pd.DataFrame(list(name_dict.items()),columns = ['team_id','team_name'])
drop_df=pd.merge(drop_df,name_df,on='team_id')
drop_df=drop_df.replace('', 0) 


#3 filter those with rebounce less than 2
test_drop_df=drop_df.loc[drop_df['rebounce'] <=2]
test_drop_list=getlist(test_drop_df,'team_id')
#sudden_drop_df=loc_list(data,'teamid',drop_com_list)
#save(sudden_drop_df,'sudden_drop_90_overlapped')
#save(drop_df,'drop_info_90')


In [61]:
### filter accounts with fluctuating use, ocassional drops in view
#data=read_csv('filtered_data')
data=fil_data
name_dict=dict(zip(data.teamid,data.company))
acc_list=getlist(data,'teamid')

#1 get those with after rise month more than 1 and less than 50%
df=drop_df
com_list=getlist(df,'team_id')
fluc_com_list=[]
for com in com_list:
    com_df=loc_one(df,'team_id',com)
    total_mon=com_df['total_months'].values[0]
    first_drop_mon=com_df['first_drop_mon'].values[0]
    after_rise_mon=com_df['after_rise_mon'].values[0]
    remaining_mon=total_mon-first_drop_mon+1
    rise_num=round(remaining_mon/2)
    if after_rise_mon>1 and after_rise_mon<=rise_num:
        fluc_com_list.append(com)
fluc_com_df=all_data.loc[all_data['teamid'].isin(fluc_com_list)]
fluc_info_df=drop_df.loc[drop_df['team_id'].isin(fluc_com_list)]
#save(fluc_info_df,'fluc_info_90')

In [ ]:
### Analyze r/s with churn
#data=read_csv('filtered_data')
data=fil_data
name_dict=dict(zip(data.teamid,data.company))
acc_list=getlist(data,'teamid')
#1 read churn accounts 
churn_info=pd.read_csv("data.csv",encoding='gb18030')
churn_acc_list=list(churn_info['account'])
churn_dict=dict()

name_key_list = list(name_dict.keys()) 
name_val_list = list(name_dict.values()) 
churn_id_list=[]
no_id_list=[]
for acc in churn_acc_list:
    try:
        churn_id=name_key_list[name_val_list.index(acc)]
        churn_id_list.append(churn_id)
        churn_dict[acc]=churn_id
    except:
        no_id_list.append(acc)
        
#2 get pct of churned accounts in low_use accounts and sudden drop acc

def get_cr(acc_list,name):
    name=name
    churn_acc=list(set(acc_list)&set(churn_id_list))
    churn_rate=len(churn_acc)/len(acc_list)
    print(name,':',len(acc_list),churn_rate)
    
get_cr(consistent_low_list,'consistent_low')
get_cr(avg_low_list,'avg_low')
get_cr(sudden_drop_list,'sudden_drop_maximum')
get_cr(drop_com_list,'sudden_drop_initial')
#get_cr(high_fluc_list,'fluc')
get_cr(acc_list,'overall')

In [63]:
### get number of times view accrosses the avg line

data=fil_data
acc_list=getlist(data,'teamid')

#1 for those accs with >= 4 months, identify drop based on first 3 months of data, put them into dicts
drop_dict=dict()
rise_dict=dict()
drop_pct_dict=dict()
rise_pct_dict=dict()
change_pct_dict=dict()

for acc in acc_list:
    acc_df=loc_one(data,'teamid',acc)
    view_list=list(acc_df['function2'])
    total_mon=len(view_list)
    avg_view= sum(view_list)/len(view_list)
    initial_view=view_list[0]
    drop=0
    rise=0
    for i in range(len(view_list)-1):
        prev_month_view=view_list[i]
        after_month_view=view_list[i+1]
        if prev_month_view < avg_view and after_month_view > avg_view:
            rise+=1
        elif prev_month_view > avg_view and after_month_view < avg_view :
            drop+=1
        drop_pct=drop/(total_mon-1)
        rise_pct=rise/(total_mon-1)
        change_pct=(drop+rise)/(total_mon-1)
    drop_dict[acc]=drop
    rise_dict[acc]=rise
    drop_pct_dict[acc]=drop_pct
    rise_pct_dict[acc]=rise_pct
    change_pct_dict[acc]=change_pct
    
drop_num=pd.DataFrame(drop_dict.items(),columns=['teamid', 'drop_num'])
rise_num=pd.DataFrame(rise_dict.items(),columns=['teamid', 'rise_num'])
drop_pct=pd.DataFrame(drop_pct_dict.items(),columns=['teamid', 'drop_pct'])
rise_pct=pd.DataFrame(rise_pct_dict.items(),columns=['teamid', 'rise_pct'])
change_pct=pd.DataFrame(change_pct_dict.items(),columns=['teamid', 'change_pct'])

fluc_df=pd.merge(drop_num,rise_num,on='teamid')
fluc_df=pd.merge(fluc_df,drop_pct,on='teamid')
fluc_df=pd.merge(fluc_df,rise_pct,on='teamid')
fluc_df=pd.merge(fluc_df,change_pct,on='teamid')

In [ ]:
### filter number of month/pct of month under the low usage line
data=fil_data
acc_list=getlist(data,'teamid')

#1 for those accs with >= 4 months, identify drop based on first 3 months of data, put them into dicts
low_dict=dict()

for acc in acc_list:
    acc_df=loc_one(data,'teamid',acc)
    view_list=list(acc_df['function2'])
    total_mon=len(view_list)
    low_mon=0
    try:
        users=user_num_dict[acc]
        for i in range(len(view_list)):
            month_view=view_list[i]
            if month_view < low_view*users:
                low_mon+=1
        low_mon_pct=low_mon/total_mon
        low_dict[acc]=low_mon_pct
    except:
        print(acc)
low_mon_pct=pd.DataFrame(low_dict.items(),columns=['teamid', 'low_mon_pct'])

In [65]:
### drop (y/n) in first 3 and last 3 months
data=fil_data
acc_list=getlist(data,'teamid')
first_drop_dict=dict()
last_drop_dict=dict()


for acc in acc_list:
    acc_df=loc_one(data,'teamid',acc)
    view_list=list(acc_df['function2'])
    total_mon=len(view_list)
    first_3=view_list[0:3]
    first_avg=sum(first_3)/3
    last_3=view_list[(total_mon-3):(total_mon+1)]
    last_avg=sum(last_3)/3
    first_drop=0
    last_drop=0
    for i in range(2):
        prev_month_view=first_3[i]
        after_month_view=first_3[i+1]
        if prev_month_view > first_avg and after_month_view < first_avg:
            first_drop+=1
    for i in range(2):
        prev_month_view=last_3[i]
        after_month_view=last_3[i+1]
        if prev_month_view > last_avg and after_month_view < last_avg:
            last_drop+=1
    first_drop_dict[acc]=first_drop
    last_drop_dict[acc]=last_drop
    
first_drop_num=pd.DataFrame(first_drop_dict.items(),columns=['teamid', 'first_drop'])
last_drop_num=pd.DataFrame(last_drop_dict.items(),columns=['teamid', 'last_drop'])
trend_df=pd.merge(first_drop_num,last_drop_num,on='teamid')

In [97]:
### qrate for all the accounts
data=fil_data
data['q_rate']=data['total_qualified']/data['function2']
q_rate=data.groupby(['teamid','company'])['q_rate'].mean().reset_index()

In [66]:
def getlog(df,col1,log_col):
    zero_list=getlist(loc_one(df,col1,0),'teamid')
    non_zero_set=set(getlist(df,'teamid'))-set(zero_list)
    df=loc_list(df, 'teamid',non_zero_set)
    df2=df.copy(deep=True)
    df2[log_col]=np.log10(df[col1])
    return df2
#avg_view=getlog(avg_view,'avg_user_view','log_avg_view')
#avg_view2['log_avg_view']=np.log10(avg_view2['avg_user_view'])

In [102]:
# Model 2
#raw_data=read_csv('training_data')
raw_data=fil_data
train_data=pd.DataFrame()
team_id=getlist(raw_data,'teamid')
train_data['teamid']=team_id

#3.average view
avg_view=raw_data.groupby('teamid')['function2'].mean().reset_index().rename(columns={'function2':'avg_view'})
avg_view=getlog(avg_view,'avg_view','log_avg_view')
train_data=pd.merge(train_data,avg_view[['teamid','log_avg_view']],on='teamid')


# s.d. of view
std_view=raw_data.groupby('teamid')['function2'].std().reset_index().rename(columns={'function2':'std'})
std_view=getlog(std_view,'std','log_std')
train_data=pd.merge(train_data,std_view[['teamid','log_std']],on='teamid')


#1. first month of usage 
#first_mon=raw_data.groupby('teamid')['function2'].first().reset_index().rename(columns={'function2':'first_month'})
#first_mon=getlog(first_mon,'first_month','log_first_month')
#train_data=pd.merge(train_data,first_mon[['teamid','first_usage']],on='teamid')
#train_data=pd.merge(train_data,first_mon[['teamid','log_first_month']],on='teamid')

#2.last 1 month of usage 
last_mon=raw_data.groupby('teamid')['function2'].last().reset_index().rename(columns={'function2':'last_month'})
last_mon=getlog(last_mon,'last_month','log_last_month')
train_data=pd.merge(train_data,last_mon[['teamid','log_last_month']],on='teamid')

#4.industry 5.segments 
segment_info=read_csv('teamid with names')
smb_list=getlist(loc_one(segment_info,'segment','SMB'),'teamid')
mm_list=getlist(loc_one(segment_info,'segment','MM'),'teamid')
ent_list=getlist(loc_one(segment_info,'segment','ENT'),'teamid')
train_data['SMB']=0
train_data['MM']=0
train_data.loc[train_data['teamid'].isin(smb_list),'SMB']=1
train_data.loc[train_data['teamid'].isin(mm_list),'MM']=1

#6 pct of times it crosses the the avg line(fluctuation pattern)
train_data=pd.merge(train_data,fluc_df[['teamid','drop_num','rise_num','change_pct']],on='teamid')

#7.whether there is a sudden dorp
train_data['sudden_drop']=0
train_data.loc[train_data['teamid'].isin(sudden_drop_list),'sudden_drop']=1

#8.number of month/pct of month under the low usage line
train_data=pd.merge(train_data,low_mon_pct,on='teamid')

#9 number of users
user_num=pd.DataFrame(user_num_dict.items(),columns=['teamid', 'num_of_users'])
user_num['log_num_of_users']=np.log10(user_num['num_of_users'])
train_data=pd.merge(train_data,user_num[['teamid','log_num_of_users']],on='teamid')

# drop trend in first few and last few months
#train_data=pd.merge(train_data,trend_df,on='teamid')

#10.number of months 
#num_of_mon=raw_data.groupby('teamid')['function2'].count().reset_index().rename(columns={'function2':'num_of_mon'})
#train_data=pd.merge(train_data,num_of_mon,on='teamid')

#11 q_rate 
train_data=pd.merge(train_data,q_rate[['teamid','q_rate']],on='teamid')


#12 whethter it has churned
train_data['churned']='non_churned'
train_data.loc[train_data['teamid'].isin(churn_id_list),'churned']='churned'

train_data=train_data.dropna()
#save(train_data,'train_data')

In [103]:
total=len(train_data.columns)
#split a validation dataset
array=train_data.values
num=total-1
X=array[:,1:num]
y=array[:,num]
X_train,X_validation,Y_train,Y_validation=train_test_split(X,y,test_size=0.20,random_state=1)

In [104]:
cols=list(train_data.columns)
cols.remove('SMB')
cols.remove('MM')
cols.remove('sudden_drop')
for i in range(len(cols)-2):
    col=cols[i+1]
    plot_df=train_data[['teamid',col]]
    #plot_df.plot(kind='box', subplots=True, layout=(1,1), sharex=False, sharey=False)
    #plot_df.hist()

In [ ]:
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))
# evaluate each model in turn
results = []
names = []
for name, model in models:
    kfold = StratifiedKFold(n_splits=5, random_state=1, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))

In [ ]:
#build model and check accuracy 
model = (LogisticRegression(solver='liblinear', multi_class='ovr'))
kfold=StratifiedKFold(n_splits=5,random_state=1,shuffle=True)
cv_result=cross_val_score(model,X_train,Y_train,cv=kfold,scoring='accuracy')
#print(cv_result.mean(), cv_result.std())

#make prediction
model.fit(X_train,Y_train)
predictions = model.predict(X_validation)

# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

In [ ]:
print(model.coef_,model.intercept_)
importance = model.coef_[0]
# summarize feature importance
for i,v in enumerate(importance):
    print('Feature: %0d, Score: %.5f' % (i,v))

In [ ]:
# Feature Extraction with RFE
from sklearn.feature_selection import RFE\
# feature extraction
model = (LogisticRegression(solver='liblinear', multi_class='ovr'))
rfe = RFE(model, 5)
fit = rfe.fit(X_train,Y_train)
print("Num Features: %d" % fit.n_features_)
print("Selected Features: %s" % fit.support_)
print("Feature Ranking: %s" % fit.ranking_)
rfe.fit(X_train,Y_train)
predictions = rfe.predict(X_validation)

# Evaluate predictions
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

In [251]:
# Model 1
raw_data=read_csv('training_data')
train_data=pd.DataFrame()
team_id=getlist(raw_data,'teamid')
train_data['teamid']=team_id

# average view
avg_view=raw_data.groupby('teamid')['function2'].mean().reset_index().rename(columns={'function2':'avg_view'})
avg_view=getlog(avg_view,'avg_view','log_avg_view')
train_data=pd.merge(train_data,avg_view2[['teamid','log_avg_view']],on='teamid')


# s.d. of view
std_view=raw_data.groupby('teamid')['function2'].std().reset_index().rename(columns={'function2':'std'})
std_view=getlog(std_view,'std','log_std')
train_data=pd.merge(train_data,std_view[['teamid','log_std']],on='teamid')


#1. first month of usage 
first_mon=raw_data.groupby('teamid')['function2'].first().reset_index().rename(columns={'function2':'first_month'})
first_mon=getlog(first_mon,'first_month','log_first_month')
#train_data=pd.merge(train_data,first_mon[['teamid','first_usage']],on='teamid')
train_data=pd.merge(train_data,first_mon[['teamid','log_first_month']],on='teamid')

#2.last 1 month of usage 
#last_mon=raw_data.groupby('teamid')['function2'].last().reset_index().rename(columns={'function2':'last_month'})
#last_mon=getlog(last_mon,'last_month','log_last_month')
#train_data=pd.merge(train_data,last_mon[['teamid','log_last_month']],on='teamid')

#4.industry 5.segments 
segment_info=read_csv('teamid with names')
smb_list=getlist(loc_one(segment_info,'segment','SMB'),'teamid')
mm_list=getlist(loc_one(segment_info,'segment','MM'),'teamid')
ent_list=getlist(loc_one(segment_info,'segment','ENT'),'teamid')
train_data['SMB']=0
train_data['MM']=0
train_data.loc[train_data['teamid'].isin(smb_list),'SMB']=1
train_data.loc[train_data['teamid'].isin(mm_list),'MM']=1

#6 number of times it crosses the the avg line(fluctuation pattern)
train_data=pd.merge(train_data,fluc_df[['teamid','drop_num','rise_num']],on='teamid')

#7.whether there is a sudden dorp
train_data['sudden_drop']=0
train_data.loc[train_data['teamid'].isin(sudden_drop_list),'sudden_drop']=1

#8.number of month/pct of month under the low usage line
train_data=pd.merge(train_data,low_mon_pct,on='teamid')

#9 number of users
user_num=pd.DataFrame(user_num_dict.items(),columns=['teamid', 'num_of_users'])
user_num['log_num_of_users']=np.log10(user_num['num_of_users'])
train_data=pd.merge(train_data,user_num[['teamid','log_num_of_users']],on='teamid')


#10.number of months 
#num_of_mon=raw_data.groupby('teamid')['function2'].count().reset_index().rename(columns={'function2':'num_of_mon'})
#train_data=pd.merge(train_data,num_of_mon,on='teamid')

#11 whethter it has churned
train_data['churned']='non_churned'
train_data.loc[train_data['teamid'].isin(churn_id_list),'churned']='churned'

train_data=train_data.dropna()

In [48]:
#淘汰的Model
raw_data=read_csv('training_data')
train_data=pd.DataFrame()
team_id=getlist(raw_data,'teamid')
train_data['teamid']=team_id

#3.average view
avg_view=raw_data.groupby('teamid')['function2'].mean().reset_index().rename(columns={'function2':'avg_view'})
user_num=pd.DataFrame(user_num_dict.items(),columns=['teamid', 'num_of_users'])
avg_view=pd.merge(avg_view,user_num,on='teamid')
avg_view['avg_user_view']=avg_view['avg_view']/avg_view['num_of_users']

avg_view=getlog(avg_view,'avg_user_view','log_avg_view')
train_data=pd.merge(train_data,avg_view[['teamid','log_avg_view']],on='teamid')

# s.d. of view
#std_view=raw_data.groupby('teamid')['function2'].std().reset_index().rename(columns={'function2':'std'})
#train_data=pd.merge(train_data,std_view[['teamid','std']],on='teamid')


#1. first month of usage 
first_mon=raw_data.groupby('teamid')['function2'].first().reset_index().rename(columns={'function2':'first_month'})
#first_mon=pd.merge(first_mon,user_num,on='teamid')
first_mon=pd.merge(first_mon,avg_view,on='teamid')
first_mon['avg_first_view']=first_mon['first_month']/first_mon['num_of_users']
#first_mon['first_usage']=first_mon['avg_first_view']/first_mon['avg_user_view']
first_mon=getlog(first_mon,'avg_first_view','log_first_view')

#train_data=pd.merge(train_data,first_mon[['teamid','first_usage']],on='teamid')
train_data=pd.merge(train_data,first_mon[['teamid','log_first_view']],on='teamid')

#2.last 1 month of usage 
last_mon=raw_data.groupby('teamid')['function2'].last().reset_index().rename(columns={'function2':'last_month'})
last_mon=pd.merge(last_mon,avg_view,on='teamid')
last_mon['avg_last_view']=last_mon['last_month']/last_mon['num_of_users']
#last_mon['last_usage']=last_mon['avg_last_view']/last_mon['avg_user_view']
last_mon=getlog(last_mon,'avg_last_view','log_last_view')


In [ ]:
#----------------------------------------------------------------------------------------------------------------------------------------

In [124]:
###PLOT GRAPHS
#1 split into companies
def plotlines(df):
    team_id=getlist(df,'teamid')
    for i in range(len(team_id)):
        comid=team_id[i]
        com_name=name_dict[comid]
        try:
            user_num=user_num_dict[comid]
            data=loc_one(df,'teamid',comid)
            graph_data=data[['date','function2']]
            graph_data=graph_data.set_index('date')
            print(com_name,":",user_num,"users")
            graph_data.plot()
            plt.legend(bbox_to_anchor=(1.05, 1),loc='upper left', borderaxespad=0.)
            plt.title(com_name,fontsize=24)
            plt.show()
        except:
            print(user_num)

In [117]:
def plot_churn(acc_list):
    churn_acc=list(set(acc_list)&set(churn_id_list))
    churn_df=loc_list(fil_data,'teamid',churn_acc)
    plotlines(churn_df)

def plot_non_churn(acc_list):
    churn_acc=list(set(acc_list)-set(churn_id_list))
    churn_df=loc_list(data,'teamid',churn_acc)
    plotlines(churn_df)

In [ ]:
get_cr(consistent_low_list,'consistent_low')
get_cr(avg_low_list,'avg_low')
get_cr(sudden_drop_list,'sudden_drop_maximum')
get_cr(drop_com_list,'sudden_drop_initial')

In [ ]:
#1 graphs for churned
plot_data=read_csv('filtered_data8')
churn_df=loc_list(plot_data,'teamid',churn_id_list)
churn_list=getlist(churn_df,'teamid')
test_list=list(set(churn_list)-set(consistent_low_list)-set(avg_low_list)-set(sudden_drop_list)-set(drop_com_list))
test_df=loc_list(plot_data,'teamid',test_list)
plotlines(test_df)

In [ ]:
churn_others=list(set(churn_id_list)-set(sudden_drop_list))
churn_others_df=loc_list(data,'teamid',churn_others)
plotlines(churn_others_df)

In [73]:
### Analyze sudden drop pattern & fluc pattern

sudden_drop_info=drop_df.loc[drop_df['team_id'].isin(drop_com_list)]
fluc_info=drop_df.loc[drop_df['team_id'].isin(fluc_com_list)]

df=fluc_info

#1 check if there is a common trend in the number of month of drop
drop_month_grp=sudden_drop_info.groupby('first_drop_mon')['team_name'].count().reset_index()

#2 check if there is a common trend in the actual month of drop
drop_date_grp=sudden_drop_info.groupby('first_drop_date')['team_name'].count().reset_index()

#3 check id there is common trend in after rise month
after_rise_grp=sudden_drop_info.groupby('after_rise_mon')['team_name'].count().reset_index()

#3 plot graph
df=sudden_drop_df[['date','teamid','function2']]
df = df.pivot(index='date', columns='teamid', values='function2')
#df.plot()
#plt.legend(bbox_to_anchor=(1.05, 1),loc='upper left', borderaxespad=0.)

In [ ]:
#4 analyse churn rate for different drop month
df=drop_month_grp
col='first_drop_mon'
for i in range(len(df[col])):
    num=df[col][i]
    four_month_df=sudden_drop_info.loc[sudden_drop_info[col]==num]
    four_month=four_month_df['team_id']
    test=sudden_drop_df.loc[sudden_drop_df['teamid'].isin(four_month)]
    #save(test,'test')
    print(num)
    try:
        get_cr(test)
    except:
        print('error')

In [ ]:
#5 deep dive for churn reason, combine all info into 1 df
churn_drop_df=sudden_drop_info.loc[sudden_drop_info['team_id'].isin(churn_drop)]
churn_drop_df=churn_drop_df.rename(columns={'team_id':'teamid'})
churn_drop_info=pd.merge(churn_drop_df,churn_df,on='teamid')

In [31]:
all_data=read_csv('month_all')
### THESE ARE MANUAL WORK 
#identify possible debook accounts and accounts with low months

#1 clean churn data,filter those which may be debook accounts - filter those with usage less than 12 months
churn_info=pd.read_csv("data2.csv",encoding='gb18030')
churn_acc_list=getlist(churn_info,'account')

churn_df=loc_list(all_data,'company',churn_acc_list)
churn_count=churn_df.groupby('company')['function2'].count().reset_index()
#save(churn_count,'churn_test')

#2 filter those with months less than 12
churn_count=churn_count.loc[churn_count['function2']<12]
#conclusion: none of the churn accounts are debook

#3.1 clean accounts with missing month data, that are non churned and non aug account
all_acc_list=list(all_data['company'].drop_duplicates())
all_acc_id=list(all_data['teamid'].drop_duplicates())
non_churn_list=list(set(all_acc_list)-set(churn_acc_list))
data=all_data.loc[all_data['company'].isin(non_churn_list)]

last_date=data.groupby('teamid')['date'].last().reset_index()
aug_df=last_date.loc[last_date['date']== '2020/8/31']
aug_list=list(aug_df['teamid'].drop_duplicates())
non_aug_list=list(set(all_acc_id)-set(aug_list))
data=data.loc[data['teamid'].isin(non_aug_list)]

#3.2 clean new accounts 
data_count=fil_data.groupby('teamid')['function2'].count().reset_index()
data_missing=data_count.loc[data_count['function2']<13]
missing_acc_list=list(data_missing['teamid'].drop_duplicates())
data_low_month=data.loc[data['teamid'].isin(missing_acc_list)]

###filter accs with records of only a few months 

def filtermons(df,num,with_aug):
#1 filter those with end date in aug as they are not applicable
    all_acc=getlist(df,'teamid')
    last_date=df.groupby('teamid')['date'].last().reset_index()
    aug_df=loc_one(last_date,'date','2020/8/31')
    aug_list=getlist(aug_df,'teamid')
    if with_aug == 'without_aug':
        filter_list=list(set(all_acc)-set(aug_list))
    elif with_aug == 'with_aug':
        filter_list=all_acc
    elif with_aug == 'only_aug':
        filter_list=aug_list

#2 count the number of months for filtered accounts
    data=loc_list(df,'teamid',filter_list)
    num_of_mon=data.groupby('teamid')['date'].count().reset_index()
    low_mon=num_of_mon.loc[num_of_mon['date'] < num]
    low_mon_acc=getlist(low_mon,'teamid')

#3 delete these accounts from all_data
    normal_mon_acc=list(set(all_acc)-set(low_mon_acc))
    return normal_mon_acc

In [33]:
### Data cleaning
#1 remove debook and other problematic accounts
all_data=read_csv('month_all')
all_acc_list=getlist(all_data,'teamid')
name_dict=dict(zip(all_data.teamid,all_data.company))
del_acc_df=read_excel('problematic_acc','others')
del_acc_list=getlist(del_acc_df,'teamid')
non_del_list=list(set(all_acc_list)-set(del_acc_list))
data=loc_list(all_data,'teamid',non_del_list)
data=data.copy(deep=True)
data['date']=pd.to_datetime(data['date'])


#2.1 fill in accounts with missing month data: read the service start date and end date
service_date_df=read_excel('service_term','without_debook')
service_date_df['Service Start date']=pd.to_datetime(service_date_df['Service Start date'])
service_date_df['Service End Date']=pd.to_datetime(service_date_df['Service End Date'])

earlist_date_df=service_date_df.groupby(['account','BMC ID'])['Service Start date'].min().reset_index()
lastest_date_df=service_date_df.groupby(['account','BMC ID'])['Service End Date'].max().reset_index()
date_range_df=pd.merge(earlist_date_df,lastest_date_df,on=['account','BMC ID'])
#adjust the start and end to jan 2019 and aug 2020
start_date=datetime(2019, 1, 1)
condition1=date_range_df['Service Start date']<start_date
date_range_df.loc[condition1, 'Service Start date'] = start_date
end_date=datetime(2020, 8, 31)
condition2=date_range_df['Service End Date']>end_date
date_range_df.loc[condition2, 'Service End Date'] = end_date


#2.2 compare the accounts, fill in those with missing months
fil_data=pd.DataFrame()
acc_id_list=getlist(data,'teamid')
unfound_acc_list=list()
for acc in acc_id_list:
    acc_df=loc_one(data,'teamid',acc)
    try:
        date_df=loc_one(date_range_df,'BMC ID',acc)
        #first_date=list(acc_df['date'])[0]
        start_date=list(date_df['Service Start date'])[0]
        end_date=list(date_df['Service End Date'])[0]
        r=pd.date_range(start=start_date,end=end_date,freq='M')
        acc_df=acc_df.set_index('date').reindex(r).fillna(0.0).rename_axis('date').reset_index()
        acc_df.loc[:,'teamid']=acc
        acc_df.loc[:,'team_subscription_poc_company']=name_dict[acc]
    except:
        unfound_acc_list.append(acc)
        acc_df=acc_df.reset_index()
    fil_data=fil_data.append(acc_df,sort=True)
    
#3 filter out the accounts with usage data less than 3 months 
#acc_list1=filtermons(fil_data,6,'with_aug')
#acc_list2=filtermons(fil_data,12,'only_aug')
#normal_acc_list=list(set(acc_list1)&set(acc_list2))
#fil_data=loc_list(fil_data,'teamid',normal_acc_list)


#4 clean new accounts
last_date=fil_data.groupby('teamid')['date'].last().reset_index()
aug_list=getlist(last_date.loc[last_date['date']== '2020/8/31'],'teamid')
data_count=loc_list(fil_data,'teamid',aug_list).groupby('teamid')['function2'].count().reset_index()
data_new=data_count.loc[data_count['function2']<13]
new_acc_list=getlist(data_new,'teamid')
all_acc_list=getlist(fil_data,'teamid')
old_acc_list=set(all_acc_list)-set(new_acc_list)
fil_data=fil_data.loc[fil_data['teamid'].isin(old_acc_list)]
fil_acc_list=getlist(fil_data,'teamid')


#5 clean accounts with less than 12 months 
full_acc_list=filtermons(fil_data,12,'with_aug')
fil_data=loc_list(fil_data,'teamid',full_acc_list)

fil_acc_list=getlist(fil_data,'teamid')
save(fil_data,'filtered_data8')